# Step 4: IRFs and DL3 scripts

In this notebook, we will run magic-cta-pipe scripts on a small data sample. Due to time constraints, we are unable to run the pipeline on all the files needed to produce the main plots (e.g. SED, light curves), so we will provide you a complete dataset to get 'nice' plots and a few MCs and data *.h5* files to run the pipeline.
You have to provide the MCP scripts' path and filenames where you want to save logging information.

In [1]:
import glob
import logging
import os
import subprocess
import sys
log = logging.getLogger()

### Instrument Response Function  

Here we will create point-like IRFs (using ring-wobble MC test sample, applying optimal gammaness and $\theta$ cuts).
The IRFs (e.g.: effective area and energy migration matrix,...) will be produced in the fitz.gz format which is then used by gammapy.
If gamma diffuse test sample is provided it is possible to calculate the offset-dependent IRFs (Full-enclosure). To get the point-like IRFs the  **fov_offset_bins** should be set to a single bin.

We can run the script directly in the terminal with:

$ python lst1_magic_create_irf.py -g yourfile -o Path1 -c configfile

with the following options:

-g: input DL2 gamma file (test sample)

-o: output directory (to save IRFs)

-c: configuration file:

In the configuration file, we have to set the parameters as shown below:

1) IRF **quality_cuts**: "disp_diff_mean < 0.22", which represent the standard quality cut on DISPs.
2) **event_type**: "software", where the options are "hardware" which stands for a hardware trigger between MAGIC and LST-1; "software", which stands for any 2- or 3-telescope combinations except MAGIC-I + MAGIC-II; "software_only_3tel", which stands only for the combo of the type MI + MII+ LST-1; and "magic_only".
3) **cut_type**: "dynamic". The options are "global", for a cut that is not energy dependent ; "dynamic", for a cut evaluated in every energy bin according to the given gamma and theta efficiencies.

In [2]:
%%capture
os.system("sed -n '199,247p' /fefs/aswg/workspace/2023_joint_analysis_school/IRF_and_DL3/input/config_dyn.yaml")




create_irf:
    quality_cuts: "disp_diff_mean < 0.22"
    event_type: "software"  # select "software", "software_only_3tel", "magic_only" or "hardware"
    weight_type_dl2: "intensity"  # select "simple", "variance" or "intensity"
    obs_time_irf: "50 h"  # used when creating a background HDU

    energy_bins:  # log space
        start: "0.01 TeV"
        stop: "1000 TeV"
        n_edges: 26

    migration_bins:  # log space
        start: 0.2
        stop: 5
        n_edges: 31

    fov_offset_bins:  # linear space, used for diffuse MCs
        start: "0 deg"
        stop: "1 deg"
        n_edges: 2

    source_offset_bins:  # linear space, used when creating PSF HDU
        start: "0 deg"
        stop: "1 deg"
        n_edges: 101

    bkg_fov_offset_bins:  # linear space, used when creating background HDU
        start: "0 deg"
        stop: "10 deg"
        n_edges: 21

    gammaness:
        cut_type: "dynamic"  # select "global" or "dynamic"
        global_cut_value: 0.8  # 

Let's start by setting up the configuration and data paths:

In [3]:
#dir_irf='/fefs/aswg/workspace/federico.dipierro/magic-sw-sc2023-data/IRF'
dir_irf=('....../IRF')
f=open(f'{dir_irf}/IRF.log','w')

scripts=('/fefs/aswg/software/virtual_env/ctasoft/magic-cta-pipe/magicctapipe/scripts/lst1_magic/')
input_gamma=('/fefs/aswg/workspace/2023_joint_analysis_school/IRF_and_DL3/input/input_step_4/DL2/gamma/*.h5')
config=('/fefs/aswg/workspace/2023_joint_analysis_school/IRF_and_DL3/input/config_dyn.yaml')

cd to the scripts directory to launch them

In [4]:
cd $scripts

/fefs/aswg/software/virtual_env/ctasoft/magic-cta-pipe/magicctapipe/scripts/lst1_magic


Lines to get files from the data directory

In [5]:
input_file_gamma = glob.glob(input_gamma)
input_file_gamma.sort()

Here we use python subprocess.run to run the script and also get a log file

In [6]:
for input_file in input_file_gamma: 
    x=subprocess.run(['python','lst1_magic_create_irf.py', f'-g{input_file}', f'-o{dir_irf}',\
        f'-c{config}'], stdout=f, stderr=f) 

We can check the log file by typing:

In [7]:
more $dir_irf/IRF.log

### Processing DL2 to DL3

Here we run a script that:
1) produces DL3 data (i.e.: DL2 data after gammaness and theta cuts). To get the cut values for every run it interpolates the cut tables, produced with the MC test sample, using the run average pointing coordinates
2) interpolates IRFs, in the same way, it interpolates the effective area values calculated with the MC test samples, for every run using its average pointing coordinates.
The DL3 data and IRFs are packed toghether in the output files (*.fits.gz*). 

We can run the script directly in the terminal with:

$ python lst1_magic_dl2_to_dl3.py -d yourfile -i Path1 -o Path2 -c configfile

with the followin options:

-d: input file (DL2 real data)

-i: directory where you saved the IRFs

-o: directory to save DL3 data

-c: configuration file

In the configuration file, we set:

1) `interpolation_method: "linear"`, where the options are "nearest" which uses IRFs/cuts values evaluated in the nearest gamma test node; and "linear/cubic", which interpolates IRFs/cuts values by using more than one gamma test node.
2) info on the target source

In [8]:
%%capture
os.system("sed -n '247,255p' /fefs/aswg/workspace/2023_joint_analysis_school/IRF_and_DL3/input/config_dyn.yaml")


dl2_to_dl3:
    interpolation_method: "nearest"  # select "nearest", "linear" or "cubic"
    source_name: "Crab"
    source_ra: null  #"83.633083 deg"  used when the source name cannot be resolved
    source_dec: null  #"22.0145 deg" used when the source name cannot be resolved


Let's settup the configuration and data paths:

In [9]:
#dir_dl3='/fefs/aswg/workspace/federico.dipierro/magic-sw-sc2023-data/DL3'
dir_dl3=('...../DL3')
f=open(f'{dir_dl3}/DL3.log','w')

scripts=('/fefs/aswg/software/virtual_env/ctasoft/magic-cta-pipe/magicctapipe/scripts/lst1_magic/')
input_data=('/fefs/aswg/workspace/2023_joint_analysis_school/IRF_and_DL3/input/input_step_4/DL2/real/*.h5')
config=('/fefs/aswg/workspace/2023_joint_analysis_school/IRF_and_DL3/input/config_dyn.yaml')

cd to the scripts directory to launch them

In [10]:
cd $scripts

/fefs/aswg/software/virtual_env/ctasoft/magic-cta-pipe/magicctapipe/scripts/lst1_magic


Lines to get files from gamma/proton folders 

In [11]:
input_file_data = glob.glob(input_data)
input_file_data.sort()

Here we use python subprocess.run to run the scripts and get a log file

In [12]:
for input_file in input_file_data:  
    y=subprocess.run(['python','lst1_magic_dl2_to_dl3.py', f'-d{input_file}', f'-i{dir_irf}',\
         f'-o{dir_dl3}', f'-c{config}'], stdout=f, stderr=f)        

### Creating Index files

Then we run the script below to create **Header Data Unit (HDU)** and **Observation** index files, needed by Gammapy to process DL3 data. Also these files are saved in the DL3 data directory.

In [13]:
z=subprocess.run(['python','create_dl3_index_files.py', f'-i{dir_dl3}'],stdout=f, stderr=f)        

In [14]:
more $dir_dl3/DL3.log